In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from tqdm.notebook import tqdm

In [3]:
raw_data = pd.read_excel("단지별 주소_200906.xlsx")

In [4]:
data = raw_data

data = data[["id", "단지명", "주소"]]
data = data.set_index(['id', '단지명'])

In [5]:
data.head()

,,주소
id,단지명,
6993,금천뉴타운,충청북도 청주시 상당구 금천동 77-7
6997,덕성,충청북도 청주시 상당구 금천동 81
6998,삼일,충청북도 청주시 상당구 금천동 122-15
7001,수정,충청북도 청주시 상당구 금천동 115-1
7004,양지,충청북도 청주시 상당구 금천동 70-1


In [7]:
driver = webdriver.Chrome("chromedriver")

In [8]:
wait_sec1 = 1
wait_sec2 = 10
for idx in tqdm(data.index):
    addr = data.loc[idx, "주소"].split()
    gu = addr[2]
    dong = addr[3]
    lot1 = addr[-1].split("-")[0]
    if len(addr[-1].split("-")) == 2:
        lot2 = addr[-1].split("-")[1]
    else:
        lot2 = None
    if len(addr) == 6:
        ri = addr[-2]
    else:
        ri = None
        
    driver.get("http://kras.chungbuk.go.kr/" +
"land_info/info/landprice/landprice.do")
    #시군구 입력
    WebDriverWait(driver, wait_sec2).until(EC.presence_of_element_located((By.XPATH,"//*[@id='sggnm']")))
    driver.find_element_by_xpath("//*[@id='sggnm']").send_keys("청주시" + gu)
    #동 입력
    time.sleep(wait_sec1)
    driver.find_element_by_xpath("//*[@id='umdnm']").send_keys(dong)
    #리 입력
    time.sleep(wait_sec1)
    if ri != None:
        driver.find_element_by_xpath("//*[@id='rinm']").clear()
        driver.find_element_by_xpath("//*[@id='rinm']").send_keys(ri)
    #지번 주소 입력
    time.sleep(wait_sec1)
    driver.find_element_by_xpath("//*[@id='textfield']").clear()
    driver.find_element_by_xpath("//*[@id='textfield2']").clear()
    if lot2 == None:
        driver.find_element_by_xpath("//*[@id='textfield']").send_keys(int(lot1))
    elif lot2 != None:
        driver.find_element_by_xpath("//*[@id='textfield']").send_keys(int(lot1))
        driver.find_element_by_xpath("//*[@id='textfield2']").send_keys(int(lot2))
    #검색 버튼 클릭
    time.sleep(2)
    driver.find_element_by_xpath("//*[@id='searchType3']").click()
    time.sleep(2)
    driver.find_element_by_xpath("//*[@id='searching']/a").click()
    time.sleep(2)
    for i in range(30):
        try:
            num = str(i+2)
            WebDriverWait(driver, wait_sec1).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-container']/div[2]/table/tbody/tr[" + num + "]/td[1]")))
            year = driver.find_element_by_xpath("//*[@id='main-container']/div[2]/table/tbody/tr[" + num + "]/td[1]").text
            value = driver.find_element_by_xpath("//*[@id='main-container']/div[2]/table/tbody/tr[" + num + "]/td[4]").text
            data.loc[idx, year] = value
        except:
            continue
    print(data.loc[idx])
    data.to_excel("공시지가.xlsx", encoding="euc-kr")

주소      충청북도 청주시 상당구 금천동 77-7
2020                670,800 원
2019                652,000 원
2018                625,400 원
2017                574,400 원
2016                494,500 원
2015                487,700 원
2014                453,100 원
2013                430,000 원
2012                388,000 원
2011                388,000 원
2010                377,000 원
2009                367,000 원
2008                377,000 원
2007                380,000 원
2006                380,000 원
2005                370,000 원
2004                350,000 원
2003                320,000 원
2002                330,000 원
2001                360,000 원
2000                360,000 원
1999                350,000 원
1998                350,000 원
1997                330,000 원
1996                310,000 원
1995                310,000 원
1994                280,000 원
1993                300,000 원
1992                196,000 원
1991                146,000 원
Name: (6993, 금천뉴타운), dtype: object
주소      충청북도 청주시 상당구 금천동 81
2020   

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=85.0.4183.83)
